## Optimizing the model parameters

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork()



  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/Users/jinwooahn/miniforge3/envs/torch_env/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-jhk0fkj8/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
#setting hyperparameters

learning_rate=2e-5
batch_size=64
epochs=5


In [5]:
# Add an optimization loop
# Each iteration of the optimization loop is called an epoch
# Train loop, Validation loop, Test loop

#initialize the loss function

loss_fn = nn.CrossEntropyLoss()


In [6]:
#optimization pass

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [7]:
#optimization happens in 3 steps
# 1. call optimizer.zero_grad() (reset the gradients)
# 2. loss.backwards() (back propagate)
# 3. optimizer.step() adjust the parameters by the gradients collected int the backward pass

def train_loop(dataloader, model, loss_fn, optimizer):
  size=len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    #compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)
    
    #backpropagation
    optimizer.zero_grad() #초기화
    loss.backward() #계산
    optimizer.step() #계산한 값 적용
    
    if batch % 100 == 0:
        loss, current = loss.item(), batch * len(X)
        print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
     
    with torch.no_grad(): #추론이기에 no grad
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [10]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------
loss: 1.375069 [    0/60000]
loss: 1.372066 [ 6400/60000]
loss: 1.258284 [12800/60000]
loss: 1.401643 [19200/60000]
loss: 1.190977 [25600/60000]
loss: 1.418437 [32000/60000]
loss: 1.167980 [38400/60000]
loss: 1.217244 [44800/60000]
loss: 1.316815 [51200/60000]
loss: 1.336596 [57600/60000]
Test Error: 
 Accuracy:62.1%, Avg loss: 0.019695 

Epoch 2
-----------------------
loss: 1.251915 [    0/60000]
loss: 1.264629 [ 6400/60000]
loss: 1.142221 [12800/60000]
loss: 1.308963 [19200/60000]
loss: 1.096941 [25600/60000]
loss: 1.329147 [32000/60000]
loss: 1.078009 [38400/60000]
loss: 1.143494 [44800/60000]
loss: 1.233792 [51200/60000]
loss: 1.262432 [57600/60000]
Test Error: 
 Accuracy:63.6%, Avg loss: 0.018453 

Epoch 3
-----------------------
loss: 1.165745 [    0/60000]
loss: 1.189878 [ 6400/60000]
loss: 1.058239 [12800/60000]
loss: 1.242146 [19200/60000]
loss: 1.033247 [25600/60000]
loss: 1.261344 [32000/60000]
loss: 1.016805 [38400/60000]
loss: 1.090438 [448

In [11]:
# save model
# can save learned parameters with state_dict() 
torch.save(model.state_dict(), "data/model.pth")